In [54]:
import os
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xarray as xr
from IPython.display import display

warnings.filterwarnings("ignore")

plt.style.use("default_light")
plt.rcParams["figure.figsize"] = (15, 10)

FOLDER = "../results/aggregated-new/"

In [93]:
subj_topo = xr.open_dataarray(os.path.join(FOLDER, "topomaps.nc"))
subj_topo = subj_topo.stack(all=("data_type", "surrogate_type", "subject_id"))

stats = pd.read_csv(os.path.join(FOLDER, "stats.csv"))

# compute correlations  between topographies
all_corrs = []

for alg1 in subj_topo["decomposition_type"]:
    for alg2 in subj_topo["decomposition_type"]:
        for map1 in subj_topo["latent map"]:
            for map2 in subj_topo["latent map"]:
                corrs = (
                    xr.corr(
                        subj_topo.sel({"decomposition_type": alg1, "latent map": map1}),
                        subj_topo.sel({"decomposition_type": alg2, "latent map": map2}),
                        dim="channels",
                    )
                    .assign_coords(
                        {"alg1": alg1, "alg2": alg2, "map1": map1, "map2": map2}
                    )
                    .expand_dims(["alg1", "alg2", "map1", "map2"])
                )
                all_corrs.append(corrs)
all_corrs = xr.combine_by_coords(all_corrs)

In [94]:
stats

,subject_id,latent map,var_GFP,var_total,template_corr,coverage,occurrence,lifespan,transition->A,transition->B,...,stationarity L=1000 t,stationarity L=1000 p-val,stationarity L=2500 t,stationarity L=2500 p-val,stationarity L=5000 t,stationarity L=5000 p-val,Hurst exp.,Hurst intercept,data_type,surrogate_type
0,sub-010002_EC,A,0.360600,0.350981,0.205518,0.248844,15.871746,15.678427,0.744872,0.083199,...,1102.350956,1.843096e-17,446.454038,5.943305e-09,241.600057,1.915584e-08,0.581381,1.486639,EC,real_data
1,sub-010002_EC,B,0.360600,0.350981,0.433136,0.262348,15.415753,17.011160,0.078551,0.764957,...,1102.350956,1.843096e-17,446.454038,5.943305e-09,241.600057,1.915584e-08,0.581381,1.486639,EC,real_data
2,sub-010002_EC,C,0.360600,0.350981,0.770813,0.224988,15.255756,14.747771,0.094226,0.087968,...,1102.350956,1.843096e-17,446.454038,5.943305e-09,241.600057,1.915584e-08,0.581381,1.486639,EC,real_data
3,sub-010002_EC,D,0.360600,0.350981,0.741556,0.263820,15.923745,16.566692,0.082181,0.080180,...,1102.350956,1.843096e-17,446.454038,5.943305e-09,241.600057,1.915584e-08,0.581381,1.486639,EC,real_data
4,sub-010003_EC,A,0.376784,0.372304,0.284512,0.279612,12.139806,23.038893,0.826333,0.062772,...,1283.503674,8.774402e-33,614.377036,9.386306e-26,234.064277,1.095411e-07,0.592553,1.662613,EC,real_data
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8907,sub-010302_EC,D,0.742426,0.672732,0.993224,0.242110,16.079357,15.054726,0.061633,0.071381,...,460.500343,4.227128e-10,214.839979,4.495608e-09,143.914549,1.632717e-11,0.528828,1.873340,EC,real_data
8908,sub-010318_EC,A,0.813969,0.753620,0.918679,0.299508,18.539258,16.140237,0.752371,0.121811,...,608.403267,4.640002e-25,249.726508,3.078784e-13,159.934877,5.880982e-14,0.713591,0.712695,EC,real_data
8909,sub-010318_EC,B,0.813969,0.753620,0.921660,0.335907,18.289268,18.366320,0.077042,0.782210,...,608.403267,4.640002e-25,249.726508,3.078784e-13,159.934877,5.880982e-14,0.713591,0.712695,EC,real_data
8910,sub-010318_EC,C,0.813969,0.753620,0.915207,0.209992,13.339466,15.742129,0.093905,0.097905,...,608.403267,4.640002e-25,249.726508,3.078784e-13,159.934877,5.880982e-14,0.713591,0.712695,EC,real_data


In [59]:
# take max between maps
maps_max = all_corrs.unstack("all").squeeze().max(["map1", "map2"])
ec_mean = (
    maps_max.sel({"data_type": "EC"})
    .stack(algs=("alg1", "alg2"))
    .groupby("algs")
    .mean("subject_id")
    .unstack("algs")
    .to_series()
    .reset_index()
    .pivot(index="alg1", columns="alg2")
)
ec_std = (
    maps_max.sel({"data_type": "EC"})
    .stack(algs=("alg1", "alg2"))
    .groupby("algs")
    .std("subject_id", ddof=1)
    .unstack("algs")
    .to_series()
    .reset_index()
    .pivot(index="alg1", columns="alg2")
)
eo_mean = (
    maps_max.sel({"data_type": "EO"})
    .stack(algs=("alg1", "alg2"))
    .groupby("algs")
    .mean("subject_id")
    .unstack("algs")
    .to_series()
    .reset_index()
    .pivot(index="alg1", columns="alg2")
)
eo_std = (
    maps_max.sel({"data_type": "EO"})
    .stack(algs=("alg1", "alg2"))
    .groupby("algs")
    .std("subject_id", ddof=1)
    .unstack("algs")
    .to_series()
    .reset_index()
    .pivot(index="alg1", columns="alg2")
)

print("== EC ==")
display(ec_mean.round(3).astype(str) + " $\pm$ " + ec_std.round(3).astype(str))

print("== EO ==")
display(eo_mean.round(3).astype(str) + " $\pm$ " + eo_std.round(3).astype(str))

== EC ==


0                                        \
alg2                 AAHC                ICA                PCA   
alg1                                                              
AAHC        1.0 $\pm$ 0.0   0.68 $\pm$ 0.197  0.879 $\pm$ 0.131   
ICA      0.68 $\pm$ 0.197      1.0 $\pm$ 0.0  0.886 $\pm$ 0.111   
PCA     0.879 $\pm$ 0.131  0.886 $\pm$ 0.111      1.0 $\pm$ 0.0   
TAAHC    0.95 $\pm$ 0.118  0.698 $\pm$ 0.174  0.886 $\pm$ 0.135   
hmm     0.354 $\pm$ 0.127  0.622 $\pm$ 0.144   0.593 $\pm$ 0.15   
kmeans  0.949 $\pm$ 0.072  0.713 $\pm$ 0.174    0.92 $\pm$ 0.11   

                                                                 
alg2                TAAHC                hmm             kmeans  
alg1                                                             
AAHC     0.95 $\pm$ 0.118  0.354 $\pm$ 0.127  0.949 $\pm$ 0.072  
ICA     0.698 $\pm$ 0.174  0.622 $\pm$ 0.144  0.713 $\pm$ 0.174  
PCA     0.886 $\pm$ 0.135   0.593 $\pm$ 0.15    0.92 $\pm$ 0.11  
TAAHC       1.0 $\pm$ 0.0  0.362 $\pm$ 0.127  0.939 $\pm$ 0.125  
hmm     0.362 $\pm$ 0.127      1.0 $\pm$ 0.0  0.367 $\pm$ 0.124  
kmeans  0.939 $\pm$ 0.125  0.367 $\pm$ 0.124      1.0 $\pm$ 0.0

== EO ==


0                                        \
alg2             AAHC                ICA                PCA   
alg1                                                          
AAHC    nan $\pm$ nan      nan $\pm$ nan      nan $\pm$ nan   
ICA     nan $\pm$ nan      1.0 $\pm$ 0.0  0.859 $\pm$ 0.156   
PCA     nan $\pm$ nan  0.859 $\pm$ 0.156      1.0 $\pm$ 0.0   
TAAHC   nan $\pm$ nan  0.725 $\pm$ 0.182  0.869 $\pm$ 0.133   
hmm     nan $\pm$ nan  0.626 $\pm$ 0.163   0.613 $\pm$ 0.17   
kmeans  nan $\pm$ nan  0.736 $\pm$ 0.164  0.892 $\pm$ 0.122   

                                                                 
alg2                TAAHC                hmm             kmeans  
alg1                                                             
AAHC        nan $\pm$ nan      nan $\pm$ nan      nan $\pm$ nan  
ICA     0.725 $\pm$ 0.182  0.626 $\pm$ 0.163  0.736 $\pm$ 0.164  
PCA     0.869 $\pm$ 0.133   0.613 $\pm$ 0.17  0.892 $\pm$ 0.122  
TAAHC       1.0 $\pm$ 0.0  0.318 $\pm$ 0.131   0.941 $\pm$ 0.07  
hmm     0.318 $\pm$ 0.131      1.0 $\pm$ 0.0  0.326 $\pm$ 0.134  
kmeans   0.941 $\pm$ 0.07  0.326 $\pm$ 0.134      1.0 $\pm$ 0.0

In [92]:
rho_ec_mean = (
    all_corrs.where(all_corrs.alg1 == all_corrs.alg2, drop=True)
    .where(all_corrs.map1 != all_corrs.map2)
    .sel({"data_type": "EC"})
    .unstack("all")
    .squeeze()
    .max(["map1", "map2"])
    .mean("subject_id")
    .to_series()
    .dropna()
    .reset_index()
    .drop("alg2", axis=1)
    .rename(columns={"alg1": "algorithm", 0: "rho EC"})
    .set_index("algorithm")
)
rho_ec_std = (
    all_corrs.where(all_corrs.alg1 == all_corrs.alg2, drop=True)
    .where(all_corrs.map1 != all_corrs.map2)
    .sel({"data_type": "EC"})
    .unstack("all")
    .squeeze()
    .max(["map1", "map2"])
    .std("subject_id", ddof=1)
    .to_series()
    .dropna()
    .reset_index()
    .drop("alg2", axis=1)
    .rename(columns={"alg1": "algorithm", 0: "rho EC"})
    .set_index("algorithm")
)
rho_eo_mean = (
    all_corrs.where(all_corrs.alg1 == all_corrs.alg2, drop=True)
    .where(all_corrs.map1 != all_corrs.map2)
    .sel({"data_type": "EO"})
    .unstack("all")
    .squeeze()
    .max(["map1", "map2"])
    .mean("subject_id")
    .to_series()
    .dropna()
    .reset_index()
    .drop("alg2", axis=1)
    .rename(columns={"alg1": "algorithm", 0: "rho EO"})
    .set_index("algorithm")
)
rho_eo_std = (
    all_corrs.where(all_corrs.alg1 == all_corrs.alg2, drop=True)
    .where(all_corrs.map1 != all_corrs.map2)
    .sel({"data_type": "EO"})
    .unstack("all")
    .squeeze()
    .max(["map1", "map2"])
    .std("subject_id", ddof=1)
    .to_series()
    .dropna()
    .reset_index()
    .drop("alg2", axis=1)
    .rename(columns={"alg1": "algorithm", 0: "rho EO"})
    .set_index("algorithm")
)
rho_disp = (
    rho_ec_mean.round(3).astype(str) + " $\pm$ " + rho_ec_std.round(3).astype(str)
)
rho_disp = rho_disp.join(
    rho_eo_mean.round(3).astype(str) + " $\pm$ " + rho_eo_std.round(3).astype(str)
)
display(rho_disp)

,rho EC,rho EO
algorithm,,
AAHC,0.773 $\pm$ 0.121,NaN
ICA,0.488 $\pm$ 0.195,0.494 $\pm$ 0.187
PCA,0.0 $\pm$ 0.0,0.0 $\pm$ 0.0
TAAHC,0.737 $\pm$ 0.142,0.678 $\pm$ 0.128
hmm,0.333 $\pm$ 0.206,0.343 $\pm$ 0.2
kmeans,0.683 $\pm$ 0.179,0.607 $\pm$ 0.148


In [103]:
single_cols = [
    "var_total",
    "var_GFP",
    "mixing time",
    "entropy",
    "max entropy",
    "entropy rate",
    "excess entropy",
    "MC entropy rate",
    "AIF 1st peak [ms]",
    "Hurst exp.",
]
single_means = stats.groupby(["decomposition_type", "data_type"])[single_cols].mean()
single_stds = stats.groupby(["decomposition_type", "data_type"])[single_cols].std(ddof=1)
display(single_means.round(3).astype(str) + " $\pm$ " + single_stds.round(3).astype(str))

var_total            var_GFP  \
decomposition_type data_type                                         
AAHC               EC          0.69 $\pm$ 0.059  0.748 $\pm$ 0.064   
ICA                EC         0.345 $\pm$ 0.067  0.355 $\pm$ 0.085   
                   EO         0.375 $\pm$ 0.058  0.388 $\pm$ 0.072   
PCA                EC         0.648 $\pm$ 0.058  0.695 $\pm$ 0.064   
                   EO         0.604 $\pm$ 0.037  0.643 $\pm$ 0.038   
TAAHC              EC         0.696 $\pm$ 0.059  0.753 $\pm$ 0.064   
                   EO         0.648 $\pm$ 0.042   0.697 $\pm$ 0.04   
hmm                EC         0.165 $\pm$ 0.053   0.167 $\pm$ 0.05   
                   EO         0.155 $\pm$ 0.058  0.154 $\pm$ 0.052   
kmeans             EC         0.699 $\pm$ 0.059  0.754 $\pm$ 0.063   
                   EO          0.653 $\pm$ 0.04   0.702 $\pm$ 0.04   

                                     mixing time            entropy  \
decomposition_type data_type                                          
AAHC               EC          3.469 $\pm$ 0.418  1.379 $\pm$ 0.009   
ICA                EC          3.946 $\pm$ 0.463  1.377 $\pm$ 0.007   
                   EO          4.005 $\pm$ 0.539   1.38 $\pm$ 0.007   
PCA                EC            3.65 $\pm$ 0.45   1.13 $\pm$ 0.081   
                   EO           3.91 $\pm$ 0.532   1.183 $\pm$ 0.04   
TAAHC              EC          3.489 $\pm$ 0.409  1.382 $\pm$ 0.004   
                   EO           3.705 $\pm$ 0.54  1.382 $\pm$ 0.004   
hmm                EC         20.172 $\pm$ 8.516   0.91 $\pm$ 0.164   
                   EO           15.627 $\pm$ 7.5  0.859 $\pm$ 0.162   
kmeans             EC            3.508 $\pm$ 0.4  1.382 $\pm$ 0.005   
                   EO          3.664 $\pm$ 0.514  1.382 $\pm$ 0.013   

                                  max entropy       entropy rate  \
decomposition_type data_type                                       
AAHC               EC         1.386 $\pm$ 0.0   0.76 $\pm$ 0.065   
ICA                EC         1.386 $\pm$ 0.0  0.704 $\pm$ 0.065   
                   EO         1.386 $\pm$ 0.0    0.7 $\pm$ 0.073   
PCA                EC         1.386 $\pm$ 0.0  0.624 $\pm$ 0.066   
                   EO         1.386 $\pm$ 0.0   0.633 $\pm$ 0.07   
TAAHC              EC         1.386 $\pm$ 0.0  0.764 $\pm$ 0.064   
                   EO         1.386 $\pm$ 0.0  0.746 $\pm$ 0.072   
hmm                EC         1.386 $\pm$ 0.0   0.246 $\pm$ 0.06   
                   EO         1.386 $\pm$ 0.0  0.263 $\pm$ 0.057   
kmeans             EC         1.386 $\pm$ 0.0  0.764 $\pm$ 0.064   
                   EO         1.386 $\pm$ 0.0  0.751 $\pm$ 0.072   

                                 excess entropy    MC entropy rate  \
decomposition_type data_type                                         
AAHC               EC          0.64 $\pm$ 0.061   0.784 $\pm$ 0.07   
ICA                EC         0.686 $\pm$ 0.063  0.721 $\pm$ 0.067   
                   EO          0.691 $\pm$ 0.07  0.714 $\pm$ 0.076   
PCA                EC         0.519 $\pm$ 0.068  0.639 $\pm$ 0.067   
                   EO         0.561 $\pm$ 0.058  0.646 $\pm$ 0.072   
TAAHC              EC          0.64 $\pm$ 0.059  0.789 $\pm$ 0.069   
                   EO         0.654 $\pm$ 0.067  0.767 $\pm$ 0.078   
hmm                EC         0.671 $\pm$ 0.147  0.255 $\pm$ 0.062   
                   EO         0.603 $\pm$ 0.141  0.274 $\pm$ 0.058   
kmeans             EC         0.637 $\pm$ 0.061  0.787 $\pm$ 0.068   
                   EO         0.646 $\pm$ 0.068  0.769 $\pm$ 0.076   

                                 AIF 1st peak [ms]         Hurst exp.  
decomposition_type data_type                                           
AAHC               EC          51.192 $\pm$ 11.496   0.622 $\pm$ 0.06  
ICA                EC           52.67 $\pm$ 15.752  0.653 $\pm$ 0.067  
                   EO          57.267 $\pm$ 19.765  0.605 $\pm$ 0.051  
PCA                EC          51.961 $\pm$ 14.

In [105]:
map_cols = ["template_corr", 'coverage', 'occurrence', 'lifespan', 'distribution', 'eq. distribution']
map_means = stats.groupby(["decomposition_type", "data_type", "latent map"])[map_cols].mean()
map_stds = stats.groupby(["decomposition_type", "data_type", "latent map"])[map_cols].std(ddof=1)
display(map_means.round(3).astype(str) + " $\pm$ " + map_stds.round(3).astype(str))

template_corr           coverage  \
decomposition_type data_type latent map                                         
AAHC               EC        A           0.891 $\pm$ 0.092  0.258 $\pm$ 0.027   
                             B           0.906 $\pm$ 0.075  0.251 $\pm$ 0.029   
                             C            0.93 $\pm$ 0.066  0.245 $\pm$ 0.031   
                             D           0.911 $\pm$ 0.077  0.245 $\pm$ 0.029   
ICA                EC        A           0.467 $\pm$ 0.245  0.252 $\pm$ 0.027   
                             B           0.444 $\pm$ 0.251  0.255 $\pm$ 0.034   
                             C            0.55 $\pm$ 0.201  0.255 $\pm$ 0.029   
                             D           0.508 $\pm$ 0.265  0.238 $\pm$ 0.039   
                   EO        A            0.511 $\pm$ 0.24  0.249 $\pm$ 0.026   
                             B           0.468 $\pm$ 0.257  0.246 $\pm$ 0.025   
                             C           0.562 $\pm$ 0.237   0.25 $\pm$ 0.028   
                             D           0.488 $\pm$ 0.243  0.256 $\pm$ 0.029   
PCA                EC        A           0.573 $\pm$ 0.229  0.232 $\pm$ 0.126   
                             B           0.524 $\pm$ 0.284  0.246 $\pm$ 0.188   
                             C            0.643 $\pm$ 0.21   0.253 $\pm$ 0.14   
                             D           0.543 $\pm$ 0.317   0.27 $\pm$ 0.228   
                   EO        A            0.583 $\pm$ 0.22  0.246 $\pm$ 0.124   
                             B           0.575 $\pm$ 0.287  0.283 $\pm$ 0.164   
                             C            0.657 $\pm$ 0.23  0.296 $\pm$ 0.132   
                             D           0.432 $\pm$ 0.264  0.175 $\pm$ 0.153   
TAAHC              EC        A           0.875 $\pm$ 0.115  0.252 $\pm$ 0.023   
                             B           0.888 $\pm$ 0.101  0.253 $\pm$ 0.022   
                             C           0.918 $\pm$ 0.079  0.246 $\pm$ 0.023   
                             D           0.889 $\pm$ 0.105  0.249 $\pm$ 0.024   
                   EO        A           0.877 $\pm$ 0.097   0.25 $\pm$ 0.022   
                             B           0.886 $\pm$ 0.097  0.252 $\pm$ 0.023   
                             C           0.916 $\pm$ 0.082  0.248 $\pm$ 0.021   
                             D           0.866 $\pm$ 0.116   0.25 $\pm$ 0.024   
hmm                EC        A            0.255 $\pm$ 0.17  0.293 $\pm$ 0.274   
                             B           0.241 $\pm$ 0.164  0.237 $\pm$ 0.254   
                             C            0.329 $\pm$ 0.18  0.213 $\pm$ 0.228   
                             D           0.296 $\pm$ 0.175  0.257 $\pm$ 0.229   
                   EO        A           0.234 $\pm$ 0.152  0.257 $\pm$ 0.266   
                             B            0.23 $\pm$ 0.153  0.265 $\pm$ 0.275   
                             C            0.247 $\pm$ 0.16  0.196 $\pm$ 0.238   
                             D            0.338 $\pm$ 0.17  0.281 $\pm$ 0.251   
kmeans             EC        A            0.851 $\pm$ 0.12  0.243 $\pm$ 0.024   
                             B           0.864 $\pm$ 0.116   0.248 $\pm$ 0.02   
                             C           0.894 $\pm$ 0.104   0.25 $\pm$ 0.022   
                             D           0.883 $\pm$ 0.112  0.259 $\pm$ 0.022   
                   EO        A           0.848 $\pm$ 0.116  0.246 $\pm$ 0.019   
                             B           0.858 $\pm$ 0.119  0.244 $\pm$ 0.024   
                             C           0.892 $\pm$ 0.109  0.253 $\pm$ 0.019   
                             D           0.847 $\pm$ 0.118  0.256 $\pm$ 0.021   

                                                 occurrence  \
decomposition_type data_type latent map                       
AAHC               EC        A           14.544 $\pm$ 1.874   
                             B           14.382 $\pm$ 1.937   
                             C           13.936 $\pm$ 1.577   
   